## 0. Importing Libraries

In [34]:
import pandas as pd
import openai
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from tqdm.auto import tqdm
from uuid import uuid4
import pandas as pd
import os
import torch
from torchtext.data.utils import get_tokenizer
import dill
import re
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoModelForQuestionAnswering
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Embeddings and vector store

In [3]:
import torch
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
#locate vectorstore
vector_path = './vector_stores'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [5]:
def predict(text_str):
    text_str = text_str.lower()
    device = 'cpu'
    regex_s = re.sub("\\(.+?\\)|[\r\n|\n\r]|!", "", text_str)
    text = " ".join(regex_s.split())
    tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
    loaded_model = torch.jit.load('../question_classification/CNN.pt')
    with open('../question_classification/vocab.pkl', 'rb') as f:
        loaded_vocab = dill.load(f)
    text = torch.tensor(loaded_vocab(tokenizer(text))).to(device)
    text = text.reshape(1, -1)
    with torch.no_grad():
        output = loaded_model(text).squeeze(1)
        predicted = torch.max(output.data, 1)[1]
        return predicted.item()

In [6]:
text = "What rice cookers are available?"
predict(text)

13

In [7]:
categories = [
    'Toys_and_Games', 'Health_and_Personal_Care', 'Cell_Phones_and_Accessories', 
    'Home_and_Kitchen', 'Musical_Instruments', 'Baby_Products', 'Sports_and_Outdoors', 
    'Patio_Lawn_and_Garden', 'Video_Games', 'Pet_Supplies', 'Tools_and_Home_Improvement', 
    'Beauty_and_Personal_Care', 'Electronics', 'Automotive', 'Office_Products', 
    'Amazon_Fashion'
]

In [8]:
categories[13]

'Automotive'

In [9]:
categories[predict(text)]

'Automotive'

In [10]:
def choose_vector_store(text, size):

    category = categories[predict(text)]
    #calling vector from local
    vector_path = './vector_stores'

    from langchain.vectorstores import FAISS

    db_file_name = f"{size}/{category}"

    vectordb = FAISS.load_local(
        folder_path = os.path.join(vector_path, db_file_name),
        embeddings = embedding_model,
        index_name = f'{category}' #default index
    )
    retriever = vectordb.as_retriever()

    return retriever

### Test model

In [11]:
#locate models
model_path = './models'
if not os.path.exists(model_path):
    os.makedirs(model_path)
    print('create path done')

In [12]:
# %cd ./models
# !git clone https://huggingface.co/anas-awadalla/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0

In [24]:
def gpt2_model(temp = 0, rep = 1.5):
    model_id = 'models/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0/'

    tokenizer = AutoTokenizer.from_pretrained(
        model_id)

    tokenizer.pad_token_id = tokenizer.eos_token_id

    bitsandbyte_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bitsandbyte_config,
        device_map='cuda:0',
        load_in_8bit=True
    )

    pipe = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        max_new_tokens=100,
        model_kwargs={
            "temperature": temp,
            "repetition_penalty": rep
        }
    )

    llm = HuggingFacePipeline(pipeline=pipe)

    return llm

In [25]:
def t5_model(temp = 0, rep = 1.5):
    model_id = './models/fastchat-t5-3b-v1.0/'

    tokenizer = AutoTokenizer.from_pretrained(
        model_id)

    tokenizer.pad_token_id = tokenizer.eos_token_id

    bitsandbyte_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.float16,
        bnb_4bit_use_double_quant = True
    )

    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_id,
        quantization_config = bitsandbyte_config, #caution Nvidia
        device_map = 'auto',
        load_in_8bit = True
    )

    pipe = pipeline(
        task="text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens = 100,
        model_kwargs = {
            "temperature" : temp,
            "repetition_penalty": rep
        }
    )

    llm = HuggingFacePipeline(pipeline = pipe)

    return llm

In [35]:
def tinyroberta_model():
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    pipe = pipeline('question-answering', model=model, tokenizer=tokenizer)

    llm = HuggingFacePipeline(pipeline = pipe)

    return llm

In [26]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [27]:
def create_chain(llm, retriever):
    question_generator = LLMChain(
        llm = llm,
        prompt = CONDENSE_QUESTION_PROMPT,
        verbose = True
    )

    prompt_template = """
        Test prompt for NLP Amazon sales chatbot.
        {context}
        Question: {question}
        Answer:
        """.strip()

    PROMPT = PromptTemplate.from_template(
        template = prompt_template
    )

    PROMPT
    #using str.format 
    #The placeholder is defined using curly brackets: {} {}
    doc_chain = load_qa_chain(
        llm = llm,
        chain_type = 'stuff',
        prompt = PROMPT,
        verbose = True
    )

    memory = ConversationBufferWindowMemory(
        k=1, 
        memory_key = "chat_history",
        return_messages = True,
        output_key = 'answer'
    )

    chain = ConversationalRetrievalChain(
        retriever=retriever,
        question_generator=question_generator,
        combine_docs_chain=doc_chain,
        return_source_documents=True,
        memory=memory,
        verbose=True,
        get_chat_history=lambda h : h
    )
    
    return chain

In [28]:
def chat_answer(prompt_question, llm):
    torch.cuda.empty_cache()
    retriever = choose_vector_store(prompt_question, 100)
    chain = create_chain(llm, retriever)
    answer = chain({"question":prompt_question})

    return answer


In [29]:
prompt_question = "Can you tell me what crossbows are available?"
llm = gpt2_model()
answer = chat_answer(prompt_question, llm)
answer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Test prompt for NLP Amazon sales chatbot.
        U.S. (Source: the npd Group). all knives feature high-carbon stainless steel blades which ensure the blades retain their ultra-sharp edge longer than conventional stainless steel. Each knife features an ergonomic metallic steel handle for greater durability. Set includes: 8-inch chef, 8-inch bread, 5-1/2-inch utility, 3-1/2-inch parer, 6 4.5-Inch steaks knives, all-purpose shears, sharpening shear and storage block with angled Heel. Not dishwasher safe. Hand wash with warm water and a mild detergent; rinse and dry immediately."], "price": "None", "images": {"hi_res": ["https://m.media-amazon.com/images/I/81+3G1q1cgL._AC_SL1500_.jpg",

"Kitchen Utensils & Gadgets", "Kitchen Knives & Accessories", "Knife Block Sets"], "details": "{\"Color\": \"Red\", \"Brand\": \"Farberware\", \"B

{'question': 'Can you tell me what crossbows are available?',
 'chat_history': [],
 'answer': ' Yes, 3 crossbows available at https://www.reddit.com/r/bowderbydesigns/\n\nThank you!\n\n1410 443 11\n\nForest, OK 79086\n\n2011-06-25 19:40:03 UTC 65412\n\n\nFavorited 40\n\nFavorited 0\n\nComments 31\n\nViews 63\n\nVotes 12\n\nConsidered (43)\n\n\nNo\n\nThe user #',
 'source_documents': [Document(page_content='U.S. (Source: the npd Group). all knives feature high-carbon stainless steel blades which ensure the blades retain their ultra-sharp edge longer than conventional stainless steel. Each knife features an ergonomic metallic steel handle for greater durability. Set includes: 8-inch chef, 8-inch bread, 5-1/2-inch utility, 3-1/2-inch parer, 6 4.5-Inch steaks knives, all-purpose shears, sharpening shear and storage block with angled Heel. Not dishwasher safe. Hand wash with warm water and a mild detergent; rinse and dry immediately."], "price": "None", "images": {"hi_res": ["https://m.medi

In [31]:
prompt_question = "What is the best eye liner?"
llm = t5_model()
answer = chat_answer(prompt_question, llm)
answer

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Test prompt for NLP Amazon sales chatbot.
        {"main_category": "All Beauty", "title": "Magic Self-Adhesive Eyeliner, Waterproof Black Liquid Sticky Eyeliner for Eyelashes Extension, Long Lasting and No Glue Needed for Women Makeup", "average_rating": 3.9, "rating_number": 47, "features": ["[Easy to Use] Just like the magnetic eyeliner, it can absorb the eyelashes as long as it is drawn on the root of the eyelashes. But our liquid eyeliner does not contain magnets, nor does it require magnetic false eyelashes.", "[Save Makeup Time] Let the time to draw eyeliner replace the time to apply glue. Avoid spending a lot of time applying glue when wearing eyelashes, and also free you from glue allergies.", "[2-in-1 Function] All you need is a

it for a whole day, it can be easily removed with makeup remover."], "description": ["HOW

{'question': 'What is the best eye liner?',
 'chat_history': [],
 'answer': '<pad>  "Double-end  Eyebrow  pencil  &  Eyeliner  pen,2  in  1  Microblading  Eyebrow  Pencil  Makeup  Long  Lasting  Waterproof  &  Smudgeproof  Natural  Looking  Brows  (Dark  Brown)"\n',
 'source_documents': [Document(page_content='{"main_category": "All Beauty", "title": "Magic Self-Adhesive Eyeliner, Waterproof Black Liquid Sticky Eyeliner for Eyelashes Extension, Long Lasting and No Glue Needed for Women Makeup", "average_rating": 3.9, "rating_number": 47, "features": ["[Easy to Use] Just like the magnetic eyeliner, it can absorb the eyelashes as long as it is drawn on the root of the eyelashes. But our liquid eyeliner does not contain magnets, nor does it require magnetic false eyelashes.", "[Save Makeup Time] Let the time to draw eyeliner replace the time to apply glue. Avoid spending a lot of time applying glue when wearing eyelashes, and also free you from glue allergies.", "[2-in-1 Function] All you